In [2]:
import numpy as np
import open3d as o3d
import pathlib
import sys
import math
import itertools
import matplotlib.pyplot as plt
import copy

from collections import Counter

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Loading in Point cloud data

UPDATE:
- P003, P013 work successfully
- P013, P014, P015, P017 also work successfully, but require altering z value bounding box plane; otherwise doesn't work.
- P010, P011 doesn't work at all.
- P005, P006 works well, but requires sacrificing small details of feet

In [42]:
# Load ply file
path = "./data/P010 2022-01-25 01_36_50.ply"

def load(path: str) -> o3d.cpu.pybind.geometry.PointCloud:
    '''
        Loads a ply file provided a valid path with robust error checking
        
        Checks file path exists and is a ply file that is not empty
        
        Args:
            path: string path to a .ply file
        
        Returns:
            pcd: non-empty point cloud data object
    '''
    
    # checks if file exists
    if not pathlib.Path(path).exists(): 
        raise FileNotFoundError("File not found.")
    
    # checking correct file format
    if pathlib.Path(path).suffix != '.ply':
        raise ValueError(f"Expected a .ply file, got {pathlib.Path(path).suffix}")
    
    # check if file is not empty
    if pathlib.Path(path).stat().st_size == 0:
        raise ValueError("File is empty.")
    
    pcd = o3d.io.read_point_cloud(path)
    
    # checks pt cloud is not empty
    if not np.asarray(pcd.points).size:
        raise ValueError("Point cloud has no points.")
    
    # pts and colors must match (not too important, but mismatches may lead to incorrect results when vis or analysing)
    if np.asarray(pcd.points).shape[0] != np.asarray(pcd.colors).shape[0]:
        raise ValueError("Point cloud has mismatch between points and colors.")
    
    return pcd # should pass all error cases

pcd = load(path)

print("PLY file loaded.")
print('Shape of points', np.asarray(pcd.points).shape)
print('Shape of colors', np.asarray(pcd.colors).shape)

PLY file loaded.
Shape of points (247847, 3)
Shape of colors (247847, 3)


In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
# pcd_vis = copy.deepcopy(pcd) # need to do this to create a deep copy of pcd

# def viewClustersViaColours(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
#     '''
#         Visualising pcd coloured by cluster applying DBSCAN (Density-Based Spatial Clustering of Applications with Noise) algorithm
#         Alter cluster_dbscan() params for different results
#             eps: max dist between two samples for them to be considered same neighbourhood
#             min_points: min number of points required to form a cluster
#     '''
    
#     labels = np.array(pcd.cluster_dbscan(eps=0.03, min_points=3, print_progress=True))
#     max_label = labels.max()
#     print(f"point cloud has {max_label + 1} clusters")
#     colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1)) # avoids div error
#     colors[labels < 0] = 0 # label=-1 indicates noise
#     pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
#     o3d.visualization.draw_geometries([pcd])

# viewClustersViaColours(pcd_vis)

point cloud has 223 clusters


In [43]:
from ipywidgets import interact, interactive, fixed, interact_manual, Button, FloatSlider, FloatText
from ipywidgets import jslink as link
from IPython.display import display
import ipywidgets as widgets

# holds final state of point cloud
final_pcd = None

pcd_tmp = np.asarray(pcd.points)

min_x, min_y, min_z = pcd_tmp.min(axis=0)
max_x, max_y, max_z = pcd_tmp.max(axis=0)

def create_grid_on_plane(thresh, axis, color):
    lines = []
    colors = []
    for i in np.linspace(-1, 1, 10):  # Adjust these values as per your requirements
        # vertical lines
        start = [i, thresh, -1] if axis == 'y' else ([thresh, i, -1] if axis == 'x' else [-1, i, thresh])
        end = [i, thresh, 1] if axis == 'y' else ([thresh, i, 1] if axis == 'x' else [1, i, thresh])
        lines.append([start, end])
        colors.append(color)
        # horizontal lines
        start = [-1, thresh, i] if axis == 'y' else ([thresh, -1, i] if axis == 'x' else [i, -1, thresh])
        end = [1, thresh, i] if axis == 'y' else ([thresh, 1, i] if axis == 'x' else [i, 1, thresh])
        lines.append([start, end])
        colors.append(color)

    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.array(lines).reshape(-1, 3)),
        lines=o3d.utility.Vector2iVector(np.array([[i, i+1] for i in range(0, len(lines)*2, 2)])),
    )

    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

def view_and_adjust_threshold(thresh_x=min_x, thresh_x_dup=max_x, thresh_y=min_y, thresh_y_dup=max_y, thresh_z=min_z):
    global final_pcd 

    pcd_tmp = np.asarray(pcd.points)
    pcd_tmpc = np.asarray(pcd.colors)

    indices = (((pcd_tmp[:,0] > thresh_x) & (pcd_tmp[:,0] < thresh_x_dup)) & 
               ((pcd_tmp[:,1] > thresh_y) & (pcd_tmp[:,1] < thresh_y_dup)) & 
               (pcd_tmp[:,2] > thresh_z))
    pcd_tmp_tmp = pcd_tmp[indices]
    pcd_tmp_tmpc = pcd_tmpc[indices]

    filtered = o3d.geometry.PointCloud()
    filtered.points = o3d.utility.Vector3dVector(pcd_tmp_tmp)
    filtered.colors = o3d.utility.Vector3dVector(pcd_tmp_tmpc)
    
    final_pcd = filtered
    
    plane_x = create_grid_on_plane(thresh_x, 'x', [1, 0, 0])  # red color for x plane
    plane_y = create_grid_on_plane(thresh_y, 'y', [0, 1, 0])  # green color for y plane
    plane_z = create_grid_on_plane(thresh_z, 'z', [0, 0, 1])  # blue color for z plane
    
    plane_x_dup = create_grid_on_plane(thresh_x_dup, 'x', [1, 0, 0])  # duplicate x plane
    plane_y_dup = create_grid_on_plane(thresh_y_dup, 'y', [0, 1, 0])  # duplicate y plane
    
    ## visualise origin pt
    # origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2, origin=[0,0,0]) 
    # o3d.visualization.draw_geometries([final_pcd, origin, plane_x, plane_y, plane_z, plane_x_dup, plane_y_dup])

    o3d.visualization.draw_geometries([final_pcd, plane_x, plane_y, plane_z, plane_x_dup, plane_y_dup])

def save_pcd(b):
    if final_pcd is not None:
        o3d.io.write_point_cloud("final.ply", final_pcd)
        print("Point cloud saved!")
    else:
        print("No point cloud to save.")

save_button = Button(description="Save point cloud")
save_button.on_click(save_pcd)  



slider_x = FloatSlider(min=min_x, max=max_x, step=0.001, value=min_x, description="x")
text_x = FloatText(value=min_x, description="x")
link((slider_x, 'value'), (text_x, 'value'))

slider_x_dup = FloatSlider(min=min_x, max=max_x, step=0.001, value=max_x, description="x_dup")
text_x_dup = FloatText(value=max_x, description="x_dup")
link((slider_x_dup, 'value'), (text_x_dup, 'value'))

slider_y = FloatSlider(min=min_y, max=max_y, step=0.001, value=min_y, description="y")
text_y = FloatText(value=min_y, description="y")
link((slider_y, 'value'), (text_y, 'value'))

slider_y_dup = FloatSlider(min=min_y, max=max_y, step=0.001, value=max_y, description="y_dup")
text_y_dup = FloatText(value=max_y, description="y_dup")
link((slider_y_dup, 'value'), (text_y_dup, 'value'))

slider_z= FloatSlider(min=min_z, max=max_z, step=0.001, value=min_z, description="z")
text_z = FloatText(value=min_z, description="z")
link((slider_z, 'value'), (text_z, 'value'))

interactive(view_and_adjust_threshold, 
         thresh_x=slider_x, 
         thresh_x_dup=slider_x_dup, 
         thresh_y=slider_y, 
         thresh_y_dup=slider_y_dup, 
         thresh_z=slider_z)

display(text_x, text_x_dup, text_y, text_y_dup, text_z, interact, save_button)


FloatText(value=-1.2634514570236206, description='x')

FloatText(value=1.3225486278533936, description='x_dup')

FloatText(value=-1.425574541091919, description='y')

FloatText(value=1.4900506734848022, description='y_dup')

FloatText(value=-0.028662221506237984, description='z')

Button(description='Save point cloud', style=ButtonStyle())

In [ ]:
o3d.visualization.draw_geometries([final_pcd])

In [44]:
# note: pcd = final_pcd creates a shallow copy, only stores reference to cropped pcd so any cahnges to pcd = changes to final_pcd   
# Use deepcopy() to create an independent / deep copy

pcd = copy.deepcopy(final_pcd) 

In [37]:
o3d.visualization.draw_geometries([pcd])

# Preprocessing

In [45]:
def downsample_clean(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Performs downsampling, outlier removal and bounding box removal
    
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
    
    Returns:
        type: None 
    '''

    # voxel downsampling: reducing overall num of pts
    voxel_size = 0.01
    downsampled = pcd.voxel_down_sample(voxel_size) 
    
    # outlier cleaning
    _, ind = downsampled.remove_statistical_outlier(nb_neighbors=100, std_ratio=2.0)
    cleaned_pcd = downsampled.select_by_index(ind)
    
    # paint removal parts
    epsilon = 0.01
    points = np.asarray(cleaned_pcd.points)
    indices = np.where(np.abs(points[:, 2] < epsilon))[0]
    
    pcd_in_color = cleaned_pcd.select_by_index(indices)
    pcd_in_color.paint_uniform_color([1,1,0])
    pcd = pcd.select_by_index(indices, invert=True)
    
    o3d.visualization.draw_geometries([cleaned_pcd])

    return pcd

downsample_clean(pcd)

PointCloud with 87312 points.

In [46]:
pcd = downsample_clean(pcd)

In [47]:

def isolateLargestCluster(pcd: o3d.cpu.pybind.geometry.PointCloud, labels: np.ndarray) -> None:
    '''
    Uses DBSCAN to group and isolate the largest point cloud 
       
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
        labels (np.ndarray): Array of clusters identified by labels
    
    Returns:
        type: o3d.cpu.pybind.geometry.PointCloud
    '''

    # finding the label of largest cluster + ignoring noisy points labeled -1
    counts = Counter(labels)
    largest_cluster_label = max(counts.items(), key=lambda x: x[1] if x[0] != -1 else -1)[0]

    # get all pts / colors of largest cluster
    largest_cluster_points = np.array(pcd.points)[labels == largest_cluster_label]
    largest_cluster_colors = np.array(pcd.colors)[labels == largest_cluster_label]
    
    # new point cloud from the largest cluster pts w/ colors
    largest_cluster_pcd = o3d.geometry.PointCloud()
    largest_cluster_pcd.points = o3d.utility.Vector3dVector(largest_cluster_points)
    largest_cluster_pcd.colors = o3d.utility.Vector3dVector(largest_cluster_colors)
    
    return largest_cluster_pcd
    
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=3, print_progress=True))
pcd = isolateLargestCluster(pcd, labels)
o3d.visualization.draw_geometries([pcd])